In [1]:
import time
import torch
import numpy as np

from sklearn.neighbors import KDTree
from sentence_transformers import SentenceTransformer, util

In [2]:
!ls '../data/saved_models/SBERT/'

10000  8000  9000


In [3]:
## Carregando o SBERT:
model = SentenceTransformer('../data/saved_models/SBERT/9000')

In [4]:
# Now you can use the loaded model to encode sentences
sentence1 = '506 112 144 148 250 258 384'
emb1 = model.encode(sentence1)
sentence2 = '506 112 144 148 258 384'
emb2 = model.encode(sentence2)

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim.item())

Cosine-Similarity: 0.9790315628051758


In [5]:
# Carregando as trajs de teste:
trajs_teste = []
with open('../data/exp1-trj.t') as f:
    for line in f:
        traj_list = line.strip().split()
        trajs_teste.append(traj_list)

In [6]:
print("Quantidade de trajetórias de teste:", len(trajs_teste))

Quantidade de trajetórias de teste: 101000


In [7]:
print(trajs_teste[2]) # query par

['51', '2263', '345', '53', '120', '405', '803', '585', '692', '1566', '533', '1728', '1880', '739', '544', '226', '8']


In [8]:
print(trajs_teste[1002]) # "alvo" da query par, ou seja, a query ímpar

['51', '430', '345', '120', '856', '131', '673', '585', '233', '2200', '533', '361', '1299', '1215', '66', '588', '7', '8']


In [9]:
# As trajs de testes estão uma lista de listas, onda cada lista insterna contém uma traj tokenizada:
# [['3176', '1346', '1301', '3303'], ..., ['508', '465', '1641']] 
# Como SBERT codifica cada sentença (e.x: '508', '465', '1641') para embedding, usamos a função abaixo 
# que recebe uma traj tokenizada e a retorna em formato de sentence string:

In [10]:
def traj2str(traj):
    """
    input: ['75476610', '75466888', '75476610', '754960']
      out: '75476610 75466888 75476610 754960'
    """
    string_traj = ' '.join(traj)
    return string_traj

In [11]:
lista = ['55', '3', '104', '244']
traj2str(lista)

'55 3 104 244'

In [12]:
def get_embeddings_for_all_sentences(trajs):
    """
    Input: list of list de trajs. Trajetória formada por ids cels.
    (e.x. trajs = [['30405995', '30413746', '30421497'], ['30429247', '30429248', '30436998']])
    Outpu: embedding de cada trajetória/sentença completa (traj) fornecido diretamente pelo SBERT
    """

    t_emb = model.encode(traj2str(trajs[0]))
    list_embs = np.empty([len(trajs), t_emb.shape[0]], dtype=np.float32)

    i = 0
    total = len(trajs)
    for traj in trajs:
        list_embs[i] = model.encode(traj2str(traj))
        i += 1
      
        # Calcula a porcentagem concluída
        percent_done = (i / total) * 100
        # Exibe a porcentagem concluída
        print(f"Progresso: {percent_done:.2f}% concluído", end="\r")  # A opção `end="\r"` permite que a impressão seja substituída na mesma linha

    return list_embs

In [13]:
# Segmentando: query (trajs pares) e dbsearch (querys ímpar + 99000 outras ímpares)
query = trajs_teste[:1000] # trajs query (pares)
dbsearch = trajs_teste[1000:101000] # dbsearch trajs (as 1000 primeiras são as query ímpar)

In [14]:
print(len(query))
print(len(dbsearch))

1000
100000


In [15]:
%%time
query = get_embeddings_for_all_sentences(query)

CPU times: user 4.05 s, sys: 0 ns, total: 4.05 s
Wall time: 4.04 s


In [16]:
%%time
dbsearch = get_embeddings_for_all_sentences(dbsearch)

CPU times: user 7min 7s, sys: 458 ms, total: 7min 8s
Wall time: 7min 7s


In [17]:
print(type(query))
print(type(dbsearch))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [18]:
query[0].shape

(768,)

## Time efficiency of SBERT using KDTree

In [19]:
def knn(q, db, k):
    tree = KDTree(db)
    
    start_time = time.time()
    for i in range(len(q)):
        _, ind = tree.query([q[i]], k=k)
    end_time = time.time()
    elapsed_time = round(end_time - start_time, 2)
    print(f"Knn time: {elapsed_time} segundos, with dbsize: {len(db)}")

In [20]:
dbsizes = [20000, 40000, 60000, 80000, 100000]
for dbsize in dbsizes:
    knn(query, dbsearch[:dbsize], 50)

Knn time: 20.25 segundos, with dbsize: 20000
Knn time: 40.6 segundos, with dbsize: 40000
Knn time: 63.7 segundos, with dbsize: 60000
Knn time: 78.87 segundos, with dbsize: 80000
Knn time: 104.3 segundos, with dbsize: 100000
